In [1]:
from vpython import *
import math
from ipywidgets import widgets

#GlowScript 2.7 VPython

N =3 # N by N by N array of atoms

spacing = 1
atom_radius = 0.1*spacing

axes = [vector(1,0,0), vector(0,1,0), vector(0,0,1)]

scene.caption= """A model of a solid represented as atoms connected by interatomic bonds.

Right button drag or Ctrl-drag to rotate "camera" to view scene.
To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
  On a two-button mouse, middle is left + right.
Shift-drag to pan left/right and up/down.
Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""

class crystal:
        
    def __init__(self,  N, atom_radius, spacing, 
                 atom_color=color.white, beam_color=color.blue, beam=1):
        self.atoms = []
        self.beams = []  
        self.bonds = []
        self.make_silicon(N, spacing, atom_radius)
                           
    def make_simple_cubic(self, N, spacing, atom_radius):  
        # create the atoms for a cubic grid; 
        for z in range(-1*N,N+1,1):
            for y in range(-1*N,N+1,1):
                for x in range(-1*N,N+1,1):
                    self.make_atom(vector(x,y,z) * spacing, atom_radius)
    
    def make_face_centered_cubic(self, N, spacing, atom_radius):
        self.make_simple_cubic(N, spacing, atom_radius)
        # add atoms for the centers of the faces (face-centered cube)
        for c in range(-1*N,N+1,1):
            for b in range(-1*N,N,1):
                for a in range(-1*N,N,1):
                    self.make_atom(vector(a+0.5, b+0.5, c    ) * spacing, atom_radius)
                    self.make_atom(vector(a+0.5, c,     b+0.5) * spacing, atom_radius)
                    self.make_atom(vector(c,     a+0.5, b+0.5) * spacing, atom_radius)        

    def make_grid(self, N, spacing):
        for b in range(-1*N,N+1,1):
            for a in range(-1*N,N+1,1):
                self.make_beam(vector(a, b, -1*N)* spacing, vector(a, b, N)* spacing, 0.01)
                self.make_beam(vector(-1*N, a, b)* spacing,  vector(N, a, b)* spacing, 0.01)
                self.make_beam(vector(b, -1*N, a)* spacing, vector(b, N, a)* spacing, 0.01)
    
    def make_bonds(self):
        for atom_a in self.atoms:
            for atom_b in self.atoms :
                delta_x= abs(atom_a.pos.x - atom_b.pos.x) 
                delta_y= abs(atom_a.pos.y - atom_b.pos.y) 
                delta_z= abs(atom_a.pos.z - atom_b.pos.z) 
                if ((delta_x == 0.25 * spacing) and 
                    (delta_y == 0.25 * spacing) and
                    (delta_z == 0.25 * spacing)):
                        bond = cylinder()
                        bond.pos = atom_a.pos
                        bond.axis =(atom_b.pos-atom_a.pos)
                        bond.radius= 0.02
                        bond.color = color.yellow
                        self.bonds.append(bond)
    
    def make_silicon (self, N, spacing, atom_radius):
        # silicon has two interpenetrating face-centered cubic" primitive lattices.
        # one is shifted 0.25 spacing in all directions
        self.make_face_centered_cubic(N, spacing, atom_radius)
        for atom in self.atoms:
            atom.pos= atom.pos + vector(0.25*spacing, 0.25*spacing, 0.25*spacing)
        self.make_face_centered_cubic(N, spacing, atom_radius)
        self.make_grid(N, spacing)
        self.make_bonds()
                    
    def make_atom(self, pos, radius, atom_color=color.white):
        atom = sphere()
        atom.pos = pos
        atom.radius = radius 
        atom.color = atom_color
        self.atoms.append(atom)
           
    def make_beam(self, start, end, radius, beam_color=color.blue):
        beam = cylinder()
        beam.pos = start
        beam.axis =(end-start)
        beam.radius= radius
        beam.color = beam_color

def etch(b):
    koh_etch(spacing)
    
def koh_etch(spacing):
    def in_v_groove(pos):
        #one plane is given by -x1-x2+x3=0
        #snd plane is given by  x1-x2-x3=0
        return ( 0 - pos.x - pos.y + pos.z < 0) and (pos.x-pos.y-pos.z <0)    
        
    etch_mat=[]
    etch_mat_y=[]
    for bond in c1.bonds:

        if in_v_groove(bond.pos):
            etch_mat.append(bond)
            etch_mat_y.append(bond.pos.y)
        else:
            if in_v_groove(bond.pos+bond.axis):
                etch_mat.append(bond)
                etch_mat_y.append(bond.pos.y)
    for atom in c1.atoms:
        if in_v_groove(atom.pos):
            etch_mat.append(atom)
            etch_mat_y.append(bond.pos.y)
    for mat in etch_mat:
        mat.visible=False
    


def hide_bonds(b):
    for b in c1.bonds :
        b.visible= False
            
button=widgets.Button(description="KOH etch")
display(button)   
button.on_click(etch)

bt_hide_bonds=widgets.Button(description="hide_bonds")
display(bt_hide_bonds)   
bt_hide_bonds.on_click(hide_bonds)

c1 = crystal(N, atom_radius, spacing)

q1= vertex(pos=vector(-1*N,0, -1*N))
q2= vertex(pos=vector(0,N,  -1*N))
q3= vertex(pos=vector(N,   N, 0))
q4= vertex(pos=vector(N,   0, N))
arrQ= [q1, q2,q3, q4]

for i in range(3):
    arrQ[i].color=color.red
    arrQ[i].opacity=0.5
mid= quad(vs=arrQ)

pt=vector(-1*N,0,N)
p1= vertex(pos=vector(N,0,N))
p2= vertex(pos=vector(0,N,N))
p3= vertex(pos=vector(-1*N,N,0))
p4= vertex(pos=vector(-1*N,0,-1*N))
arrP= [p1, p2, p3, p4]
for i in range(4):
    arrP[i].color=color.green
    arrP[i].opacity=0.5
mid= quad(vs=arrP)


blaze= math.degrees((vector(1,1,1)).diff_angle(vector(1,1,-1)))
print(blaze)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='KOH etch', style=ButtonStyle())

Button(description='hide_bonds', style=ButtonStyle())

70.52877936550931


In [2]:
for atom in c1.atoms :
    atom.visible= True
    atom.color=color.white


In [2]:
from sympy import Point, Point3D, Line, Line3D, Plane
plane_1_1_1 = Plane(Point3D(N,0,N), normal_vector=(-1, -1, 1))
plane_1_1_m1 = Plane(Point3D(-1*N,0,-1*N), normal_vector=(1, -1, -1))

for atom in c1.atoms:
    rate(10)
    b = Point3D(atom.pos.x,atom.pos.y, atom.pos.z)
    proj_A= plane_1_1_1.projection(b)
    t= atom.pos - vector(proj_A.x, proj_A.y, proj_A.z)
    alfa=math.degrees(vector(-1,-1,1).diff_angle(t))
    
    proj_B= plane_1_1_m1.projection(b)
    t= atom.pos - vector(proj_B.x, proj_B.y, proj_B.z)
    beta=math.degrees(vector(1,-1,-1).diff_angle(t))
    
    if alfa>170 and beta >170:
        #atom.visible= False
        atom.visible=False
        


In [6]:
from sympy import Point, Point3D, Line, Line3D, Plane
a = Plane(Point3D(1, 1, 1), normal_vector=(1, 1, 1))
b = Point3D(1, 2, 3)
print(a.distance(b))

sqrt(3)


In [23]:
from sympy import Plane, Point, Point3D
A = Plane(Point3D(-2*N, 0, 0), normal_vector=(-1, -1, 1))
b = Point3D(-1*N,3*N,2*N)

proj=A.projection(b)
print(b-proj)
t= vector(-1*N,3*N,2*N)-vector(proj.x,proj.y,proj.z)
print(math.degrees(vector(-1,-1,1).diff_angle(t)))

print(math.degrees(vector(-1,-1,1).diff_angle(-1*t)))


Point3D(4/3, 4/3, -4/3)
180.0
0.0


In [3]:
print(vector(-1,-1,0).cross(vector(-1,0,-1)))

<1, -1, -1>


In [3]:
for b in c1.bonds :
    b.visible= False
